In [16]:
#include "TChain.h"
#include "TFile.h"
#include "TCanvas.h"
#include "TH2F.h"
#include "TLine.h"
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "RooPlot.h"
#include "RooFit.h"
#include <vector>
#include <iostream>

In [8]:
// Load TChain
TChain uncut("pippi0");
uncut.Add("out/pippi0_fall2018_in_pass2/pippi0_fall2018_in_pass2.root");
uncut.Add("out/pippi0_spring2019_in_pass2/pippi0_spring2019_in_pass2.root");

In [9]:
TTree* chain = uncut.CopyTree("Mdiphoton<0.16 && 0.115<Mdiphoton && 0.85<Mx && Mx<1.05");

In [4]:
RooRealVar Mh("Mh", "Mh", 0.4, 1.7);
Mh.setRange("fullRange", 0.4, 1.4);

RooRealVar mu("m_{0}", "mu", 0.8, 0.6, 1);
RooRealVar sigma("#sigma", "sigma", 0.06, 0.00001, 0.4);
RooRealVar p1("p1", "p1", 0, -5, 5);
RooRealVar p2("p2", "p2", 0, -5, 5);
RooRealVar p3("p3", "p3", 0, -5, 5);
RooRealVar p4("p4", "p4", 0, -5, 5);

RooRealVar N_sig("N_{sig}", "N_sig", 10000, 0, filtered->GetEntries());
RooRealVar N_bkg("N_{bkg}", "N_bkg", 10000, 0, filtered->GetEntries());

RooRealVar z("z", "z", 0, 1);
RooRealVar phi("phi", "phi", -3.14, 3.14);
RooRealVar hel("hel", "hel", -1, 1);

[#1] INFO:Eval -- RooRealVar::setRange(Mh) new range named 'fullRange' created with bounds [0.4,1.4]


In [5]:
RooGaussian sig("sig", "gaussian Fit", Mh, mu, sigma);
RooArgList pars_pol(p1, p2, p3, p4);
RooChebychev background("background", "Background", Mh, pars_pol);
RooAddPdf model_ext("model_ext", "Signal + Background", RooArgList(sig, background), RooArgList(N_sig, N_bkg));

In [17]:
std::vector<double> zbn_edges = {0.66, 0.74, 0.79, 0.82, 0.85, 0.88, 0.91, 0.93, 0.96, 1.0};
std::vector<double> phibn_edges = {-3.14, -1.047, 0.0, 1.047, 3.14};
TCanvas c2D;
c2D.SetTickx();
c2D.SetTicky();

TH2F binning("binning","binning scheme;z [GeV];#phi_{h} [rad]",100,0.6,1,100,-3.2,3.2);

chain->Draw("phi:z>>binning");

binning.SetStats(0);
binning.Draw();

for (double z : zbn_edges) {
    TLine line(z, -3.14, z, 3.14);
    line.SetLineColor(kBlack);
    line.SetLineWidth(3);
    line.Draw();
        }
for (double phi : phibn_edges){
    TLine line(0.6, phi, 1, phi);
    line.SetLineColor(kBlack);
    line.SetLineWidth(3);
    line.Draw();
        }
c2D.Draw();


cling JIT session error: Failed to materialize symbols: { (main, { _ZNSt6vectorIdSaIdEE5beginEv }) }
[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { __cxx_global_var_initcling_module_394_.3, __cxx_global_var_initcling_module_394_.4, _ZN12__cling_N53616__cling_Un1Qu344EPv, __cxx_global_var_initcling_module_394_.5, _ZN12__cling_N53611phibn_edgesE, __cxx_global_var_initcling_module_394_, _ZN12__cling_N5367binningE, __vd_init_order__cling_Un1Qu315, _Z31__fd_init_order__cling_Un1Qu314v, $.cling-module-394.__inits.0, _ZN12__cling_N5363c2DE, __cxx_global_var_initcling_module_394_.1, _ZN12__cling_N5369zbn_edgesE, __orc_init_func.cling-module-394, _GLOBAL__sub_I_cling_module_394 }) }


In [ ]:
using namespace RooFit;

void BSA_phiBinning_RooReal() {


    // Apply selection
    

    // Define variables
    

    // Define model


    // Define bin edges


    // Loop over bins
    for (size_t i = 0; i < zbn_edges.size() - 1; ++i) {
        for (size_t j = 0; j < phibn_edges.size() - 1; ++j) {
            double zmin = zbn_edges[i];
            double zmax = zbn_edges[i + 1];
            double phimin = phibn_edges[j];
            double phimax = phibn_edges[j + 1];

            TString cut = Form("z > %f && z < %f && phi > %f && phi < %f && hel == -1", zmin, zmax, phimin, phimax);
            RooDataSet neg_DS("neg_DS", "neg_DS", filtered, RooArgSet(Mh, z, phi, hel), cut);

            cut = Form("z > %f && z < %f && phi > %f && phi < %f && hel == 1", zmin, zmax, phimin, phimax);
            RooDataSet pos_DS("pos_DS", "pos_DS", filtered, RooArgSet(Mh, z, phi, hel), cut);

            model_ext.fitTo(neg_DS, Range("fullRange"), Save(), PrintLevel(-1), Extended(kTRUE));
            model_ext.fitTo(pos_DS, Range("fullRange"), Save(), PrintLevel(-1), Extended(kTRUE));

            // Optional: plot
            RooPlot* frame = Mh.frame(0.4, 1.7);
            neg_DS.plotOn(frame, MarkerSize(0.5), Name("data"));
            model_ext.plotOn(frame, LineStyle(kDashed), LineColor(kBlack), Name("totalFit"));
            model_ext.plotOn(frame, Components("background"), LineColor(kRed), Name("chebychevFit"));
            model_ext.plotOn(frame, Components("sig"), LineColor(kBlue), Name("sigFit"));

            TCanvas* c = new TCanvas();
            frame->Draw();
            c->SaveAs(Form("fit_z%d_phi%d.png", (int)i, (int)j));
        }
    }
}
